In [1]:
import requests
import pandas as pd
import numpy as np 
from pyarrow import fs
import pyarrow as pa
import pyarrow.parquet as pq
from scipy.stats import zscore
import matplotlib.pyplot as plt
from tqdm import tqdm
from helpers import *
pd.set_option('display.float_format', '{:.2f}'.format)


Data extraction from multiples files : API SNCF, API GOUV, economic regional data from web csv
first use sncf well established api to access data of stations 

In [85]:
station_geo_data = get_names_geo_data_from_sncf_api(
    endpoint_suffix="gares-de-voyageurs",
    select="""nom as nom_gare,
                           position_geographique,
                           codeinsee,
                           codes_uic as uic""",
)

station_freq_data = get_names_geo_data_from_sncf_api(
    endpoint_suffix="frequentation-gares",
    select="""total_voyageurs_2022,
                           
                           nom_gare,
                           code_uic_complet as uic """,
)

nb of stations downloaded: 2881, from table gares-de-voyageurs
nb of stations downloaded: 3010, from table frequentation-gares


In [ ]:
station_geo_data = station_geo_data.rename(
    columns={
        "position_geographique.lon": "lon_gare",
        "position_geographique.lat": "lat_gare",
    }
)
station_geo_data = station_geo_data.drop("position_geographique", axis=1)

In [ ]:
merged = station_freq_data.merge(
    station_geo_data.drop("nom_gare", axis=1), on=["uic"], how="left"
)
merged = merged[merged["total_voyageurs_2022"] > 0]

here insert code from communes and delete extract from S3!!!!!!!!!!!!!

get stat about regions

In [88]:
s3=s3_connection("aayrapetyan")
regional_stat=s3.get_tables_from_s3("clichere/diffusion/rev_dip_pop_comm.parquet")

connection successful


In [ ]:
regional_stat = regional_stat[
    (regional_stat["pop2022"] > 0)
    & (regional_stat["revmoy2022"] > 0)
    & (regional_stat["pbac2022"].notna())
]

In [90]:
fusion=merged.merge(regional_stat,left_on="codeinsee",right_on="codecommune",how="left")

In [91]:
fusion.sample(2)

,total_voyageurs_2022,nom_gare,uic,codeinsee,lon_gare,lat_gare,codecommune,nomcommune,pop2022,revmoy2022,pbac2022,psup2022
377,184840,Orgerus - Béhoust,87393447,78465,1.70,48.83,78465,ORGERUS,2303.00,26872.25,0.60,0.48
1262,2079,Base Aérienne,87699223,41194,1.68,47.32,41194,ROMORANTIN-LANTHENAY,19202.00,15276.31,0.39,0.23


In [92]:
final=fusion[fusion["pop2022"].notna()]

In [94]:
cross_table_for_distance_calculation = final[["uic", "lon_gare", "lat_gare"]].merge(
    final[["uic", "lon_gare", "lat_gare"]], how="cross"
)
(
    lat1,
    lon1,
    lat2,
    lon2,
) = (
    cross_table_for_distance_calculation["lat_gare_x"],
    cross_table_for_distance_calculation["lon_gare_x"],
    cross_table_for_distance_calculation["lat_gare_y"],
    cross_table_for_distance_calculation["lon_gare_y"],
)
cross_table_for_distance_calculation["dist_closest_station_km"] = haversine_vectorized(
    lat1, lon1, lat2, lon2
)

In [ ]:
cross_table_for_distance_calculation = cross_table_for_distance_calculation[
    cross_table_for_distance_calculation["uic_x"]
    != cross_table_for_distance_calculation["uic_y"]
]

In [ ]:
idx = cross_table_for_distance_calculation.groupby(["uic_x"])[
    "dist_closest_station_km"
].idxmin()

result = cross_table_for_distance_calculation.loc[idx].reset_index(drop=True)

In [ ]:
final = final.merge(
    result[["uic_x", "dist_closest_station_km"]].rename(columns={"uic_x": "uic"}),
    on="uic",
    how="inner",
)

In [100]:

new=final.groupby('nomcommune').agg({'nomcommune': ['count']})
new["commune"]=new.index
new=new.reset_index(drop=True)
new.columns = ["nb_stations_same_commune","nomcommune"]

In [101]:
final=final.merge(new,on="nomcommune",how="inner")

In [ ]:
french_regions=gouv_api_addresses(final)

In [279]:
final["regions"].unique()

array(['Pays de la Loire', 'Île-de-France', 'Hauts-de-France',
       "Provence-Alpes-Côte d'Azur", 'Occitanie', 'Auvergne-Rhône-Alpes',
       'Bourgogne-Franche-Comté', 'Centre-Val de Loire', 'Normandie',
       'Nouvelle-Aquitaine', 'Grand Est', 'Bretagne'], dtype=object)

In [278]:
final["regions"]=french_regions

In [280]:
final.isna().sum()

total_voyageurs_2022        0
nom_gare                    0
uic                         0
codeinsee                   0
lon_gare                    0
lat_gare                    0
codecommune                 0
nomcommune                  0
pop2022                     0
revmoy2022                  0
pbac2022                    0
psup2022                    0
dist_closest_station_km     0
nb_stations_same_commune    0
regions                     0
dtype: int64

In [288]:
final.sample(2)

,total_voyageurs_2022,nom_gare,uic,codeinsee,lon_gare,lat_gare,codecommune,nomcommune,pop2022,revmoy2022,pbac2022,psup2022,dist_closest_station_km,nb_stations_same_commune,regions
175,1469,Gudmont,87142265,52230,5.14,48.34,52230,GUDMONT-VILLIERS,234.00,19475.53,0.39,0.35,3.12,1,Grand Est
930,9781,Montreuil-Bellay,87487637,49215,-0.15,47.13,49215,MONTREUIL-BELLAY,3726.00,15218.20,0.32,0.18,16.27,1,Pays de la Loire


In [284]:
s3.from_pandas_to_parquet_store_in_s3(final,"aayrapetyan/diffusion/final_table.parquet")

In [285]:
final=s3.get_tables_from_s3("aayrapetyan/diffusion/final_table.parquet")
final

,total_voyageurs_2022,nom_gare,uic,codeinsee,lon_gare,lat_gare,codecommune,nomcommune,pop2022,revmoy2022,pbac2022,psup2022,dist_closest_station_km,nb_stations_same_commune,regions
0,40825,Abbaretz,87481614,44001,-1.52,47.55,44001,ABBARETZ,2257.00,12567.25,0.27,0.17,9.75,1,Pays de la Loire
1,177092,Achères Grand Cormier,87386052,78551,2.09,48.96,78551,SAINT-GERMAIN-EN-LAYE,39172.00,41601.99,0.80,0.64,1.97,1,Île-de-France
2,80648,Achiet-le-Grand,87342048,62005,2.78,50.13,62005,ACHIET-LE-GRAND,888.00,15813.03,0.42,0.34,4.22,1,Hauts-de-France
3,32800,Agay,87757559,83118,6.86,43.43,83118,SAINT-RAPHAEL,37114.00,26637.97,0.55,0.34,1.80,6,Provence-Alpes-Côte d'Azur
4,11325,Aigues-Mortes,87775858,30003,4.19,43.57,30003,AIGUES-MORTES,8076.00,20798.90,0.37,0.20,5.56,1,Occitanie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2803,176599,Weyersheim,87213678,67529,7.80,48.72,67529,WEYERSHEIM,3365.00,21737.75,0.40,0.26,2.10,1,Grand Est
2804,30849,Willer-sur-Thur,87182584,68372,7.07,47.84,68372,WILLER-SUR-THUR,1735.00,18620.66,0.38,0.13,1.82,1,Grand Est
2805,37020,Wimille - Wimereux,87317123,62894,1.61,50.76,62894,WIMILLE,4062.00,19524.12,0.40,0.28,4.08,1,Hauts-de-France
2806,38618,Ygos-Saint-Saturnin,87671487,40333,-0.74,43.98,40333,YGOS-SAINT-SATURNIN,1490.00,15004.68,0.46,0.28,6.68,1,Nouvelle-Aquitaine
